## ***PGMExplainer***


> Moduled: Accpeting the four GNNs (GCN+GAP, DGCNN, DIFFPOOL, and GIN)


---

In [ ]:
# Install required packages.
import os

#!pip install torch==1.7.0
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)


!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install git+https://github.com/rusty1s/pytorch_geometric.git

2.2.1+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 32.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/rusty1s/pytorch_geometric.git to /tmp/pip-req-build-d1wgt6fi
  Running command git clone --filter=blob:none --quiet https://github.com/rusty1s/pytorch_geometric.git /tmp/pip-req-build-d1wgt6fi
  Resolved https://github.com/rusty1s/pytorch_geometric.git to commit e213c297bb2aeb9ac50db258f5ab01ea11aea349
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import argparse
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from math import sqrt
import math

from torch_geometric.datasets import TUDataset

import torch as th
import torch
import torch.nn as nn
from torch import Tensor
from torch.nn.parameter import Parameter
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch.nn import Linear, LayerNorm
from sklearn import metrics
from scipy.spatial.distance import hamming
import statistics
import pandas
from time import perf_counter
from IPython.core.display import deepcopy
from torch_geometric.nn import MessagePassing
import copy
from torch.nn import ReLU, Sequential
from torch import sigmoid
from itertools import chain
from time import perf_counter
from torch_geometric.data import Data, Batch, Dataset
from functools import partial
from torch_geometric.utils import to_networkx
from torch_geometric.utils import remove_self_loops
from typing import Callable, Union, Optional
#from torch_geometric.utils.num_nodes import maybe_num_nodes
import networkx as nx
from typing import List, Tuple, Dict
from collections import Counter
import statistics
from scipy import stats
import logging
import pandas as pd
import csv
from statistics import mean


from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.loader import DataLoader
import torch_geometric.nn as gnn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
mutag_dataset = TUDataset(root='data/TUDataset', name='MUTAG')

Processing...
Done!


In [ ]:
Explainability_name = 'PGMExplainer'
Task_name = 'Graph Classification'
checkpoint_directory_Classifier = "/content/drive/My Drive/Explainability Methods/" + str(Explainability_name) + " on " + str(Task_name) + "/Model/model_classifier.pt"
DataSet_name = "MUTAG"
classifier_lr = 0.001
classifier_dropout = 0.1
classifier_weight_decay = 1e-6
classifier_bias = True

#File_Name = Model_name + " " + Explainability_name + " " + Task_name + " " + DataSet_name + " "

In [ ]:
#train_dataset, test_dataset = train_test_split(dataset, test_size=0.3, random_state=0, shuffle=True)
#print("Number of Training Graphs: ", len(train_dataset))
#print("Number of Test Graphs: ", len(test_dataset))

df = pandas.read_csv("/content/drive/My Drive/Explainability Methods/Train and Test Indexes on Graph Classification/Experimental Results/train_test_indexes.csv")

read_training_list_indexes__ = df['Train Indexes']
read_test_list_indexes__ = df['Test Indexes']
read_test_list_indexes__ = read_test_list_indexes__.dropna()
read_test_list_indexes = []
read_training_list_indexes = []
for element in read_test_list_indexes__:
    read_test_list_indexes.append(int(element))
for element in read_training_list_indexes__:
    read_training_list_indexes.append(int(element))


print(read_training_list_indexes)
print(read_test_list_indexes)

mutag_train_dataset = []
mutag_test_dataset = []
for index in read_training_list_indexes:
    mutag_train_dataset.append(mutag_dataset[index])
for index in read_test_list_indexes:
    mutag_test_dataset.append(mutag_dataset[index])


print(f'Number of training graphs: {len(mutag_train_dataset)}')
print(f'Number of test graphs: {len(mutag_test_dataset)}')

[51, 142, 143, 10, 4, 141, 125, 23, 3, 79, 28, 117, 136, 156, 83, 128, 109, 70, 31, 58, 7, 148, 14, 187, 20, 162, 123, 13, 91, 185, 169, 102, 32, 55, 119, 25, 161, 175, 52, 121, 152, 108, 113, 65, 39, 103, 131, 42, 166, 110, 160, 68, 90, 89, 64, 172, 159, 72, 170, 18, 122, 29, 179, 49, 171, 178, 9, 74, 96, 48, 181, 127, 126, 87, 12, 163, 88, 53, 94, 146, 5, 158, 16, 67, 6, 59, 164, 151, 34, 47, 54, 46, 100, 112, 93, 182, 66, 106, 124, 19, 186, 133, 45, 15, 40, 167, 174, 98, 105, 153, 61, 63, 132, 116, 43, 80, 33, 147, 165, 69, 135, 86, 76, 57, 173, 115, 138, 140, 134, 180, 95, 22, 38, 41, 24, 120, 145, 26, 21, 50, 176, 107, 78, 17, 85, 154, 60, 92, 184, 129]
[0, 1, 2, 8, 11, 27, 30, 35, 36, 37, 44, 56, 62, 71, 73, 75, 77, 81, 82, 84, 97, 99, 101, 104, 111, 114, 118, 130, 137, 139, 144, 149, 150, 155, 157, 168, 177, 183]
Number of training graphs: 150
Number of test graphs: 38


In [ ]:
BATCH_SIZE = 64

mutag_train_dataloader = DataLoader(mutag_train_dataset, batch_size=BATCH_SIZE, shuffle=False) # important to be false
mutag_test_dataloader = DataLoader(mutag_test_dataset, batch_size=1, shuffle=False)

In [ ]:
from torch_geometric.datasets import FakeDataset

num_graphs = 100
avg_num_nodes = 20
avg_degree = 15
node_feat_dim = 13
edge_feat_dim = 0
num_classes = 3

fake_dataset = FakeDataset(num_graphs=num_graphs, avg_num_nodes=avg_num_nodes, avg_degree=avg_degree, num_channels=node_feat_dim,
                           edge_dim=edge_feat_dim, num_classes=num_classes,)
Explainability_name = 'PGMExplainer'
Task_name = 'Graph Classification'
classifier_bias = True
DataSet_name = "Fake"
BATCH_SIZE = 10
classifier_lr = 0.001
classifier_dropout = 0.1
classifier_weight_decay = 1e-6

train_ratio = 0.8

fake_train_dataset = fake_dataset[:int(len(fake_dataset) * train_ratio)]
fake_test_dataset = fake_dataset[len(fake_dataset) - int(len(fake_dataset) * train_ratio):]

fake_train_dataloader = DataLoader(fake_train_dataset, batch_size=BATCH_SIZE, shuffle=False)
fake_test_dataloader = DataLoader(fake_test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
import sys
py_path = '/content/drive/MyDrive/Explainability Methods/Models/Script/'
sys.path.insert(0,py_path)

#import GCN_plus_GAP as Graph_Network
#GNN_Model = Graph_Network.GCN_plus_GAP(model_name='GCN_plus_GAP', model_level='graph', input_dim=13, hidden_dim=13, output_dim=3,
#                                       num_hid_layers=3, Bias=classifier_bias, act_fun='ReLu', Weight_Initializer=1,
#                                       dropout_rate=classifier_dropout)

#import DGCNN as dgcnn_model
#GNN_Model = dgcnn_model.DGCNN_Model(GNN_layers=[32, 32, 32, 13], num_classes=3, node_feat_size=13, mlp_act_fun='ReLu',
#                                    dgcnn_act_fun='tanh', mlp_dropout_rate=0.5, Weight_Initializer=3, Bias=False, dgcnn_k=17,
#                                    hid_channels=[16,32], conv1d_kernels=[2,5], ffn_layer_size=128, strides=[2,1])

#import DIFFPOOL as diffpool_model
#GNN_Model = diffpool_model.DIFFPOOL_Model(embedding_input_dim=13, embedding_num_block_layers=1, embedding_hid_dim=64,
#                                          new_feature_size=64, assignment_input_dim=13, assignment_num_block_layers=1,
#                                          assignment_hid_dim=64, max_number_of_nodes=256, prediction_hid_layers=[50],
#                                          concat_neighborhood=False, num_classes=3, Weight_Initializer=1, Bias=classifier_bias,
#                                          dropout_rate=0, normalize_graphsage=False, aggregation="mean", act_fun="ReLu",
#                                          concat_diffpools_outputs=True, num_pooling=1, pooling="mean")

import GIN as gin_model
GNN_Model = gin_model.GIN_Model(num_mlp_layers=4, mlp_input_dim=13, mlp_hid_dim=20, mlp_output_dim=2, num_slp_layers=2,
                                Bias=classifier_bias, mlp_act_fun="ReLu", dropout_rate=classifier_dropout, Weight_Initializer=1,
                                joint_embeddings=False)



Model_name = GNN_Model.__class__.__name__

In [ ]:
GNN_Model_Optimizer = torch.optim.Adam(GNN_Model.parameters(), lr=classifier_lr, weight_decay=classifier_weight_decay)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
def loss_calculations(preds, gtruth):
    loss_per_epoch = criterion(preds, gtruth)
    return loss_per_epoch

In [ ]:
def visualize_losses(GNN_Model_losses, epoch_history):
    GNN_Model_losses_list = torch.stack(GNN_Model_losses).cpu().detach().numpy()

    fig = plt.figure(figsize=(27,20))

    ax = plt.subplot2grid((3, 1), (0, 0), colspan=1)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(" Loss in Epoch: " + str(epoch_history))

    ax.plot(GNN_Model_losses_list, color='r')

    #plt.savefig('/content/drive/My Drive/Explainability Methods/'+str(Explainability_name)+' on ' + str(Task_name) + '/Experimental Results/' + File_Name + 'Loss_til_epoch_{:04d}.png'.format(epoch_history))
    plt.show()


In [ ]:
def train_step(data):
    GNN_Model_loss_batch = []
    Pred_Labels = []
    Real_Labels = []

    GNN_Model.train()
    GNN_Model.zero_grad()
    for batch_of_graphs in data:
        if GNN_Model.__class__.__name__ == "GCN_plus_GAP":
            Output_of_Hidden_Layers, pooling_layer_output, ffn_output, soft = GNN_Model(batch_of_graphs)
            batch_loss = loss_calculations(soft, batch_of_graphs.y)
            Pred_Labels.extend(soft.argmax(dim=1).detach().tolist())

        elif GNN_Model.__class__.__name__ == "DGCNN_Model":
            final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, output_h1, dropout_output_h1, output_h2, softmaxed_h2 = GNN_Model(batch_of_graphs, None)
            batch_loss = loss_calculations(softmaxed_h2, batch_of_graphs.y)
            Pred_Labels.extend(softmaxed_h2.argmax(dim=1).detach().tolist())

        elif GNN_Model.__class__.__name__ == "DIFFPOOL_Model":
            concatination_list_of_poolings, prediction_output_without_soft, prediction_output = GNN_Model(batch_of_graphs, None)
            Pred_Labels.extend(prediction_output.argmax(dim=1).detach().tolist())
            batch_loss = loss_calculations(prediction_output, batch_of_graphs.y)

        elif GNN_Model.__class__.__name__ == "GIN_Model":
            mlps_output_embeds, mlp_outputs_globalSUMpooled, lin1_output, lin1_output_dropouted, lin2_output, lin2_output_softmaxed = GNN_Model(batch_of_graphs, None)
            batch_loss = loss_calculations(lin2_output_softmaxed, batch_of_graphs.y)
            Pred_Labels.extend(lin2_output_softmaxed.argmax(dim=1).detach().tolist())
        else:
            raise Exception("We cover GCN_plus_GAP, DGCNN, DIFFPOOL, and GIN.")

        Real_Labels.extend(batch_of_graphs.y.detach().tolist())
        GNN_Model_loss_batch.append(batch_loss)

        batch_loss.backward()
        GNN_Model_Optimizer.step()

    return torch.mean(torch.tensor(GNN_Model_loss_batch)), metrics.accuracy_score(Real_Labels, Pred_Labels)

In [ ]:
GNN_Model_training_Acc_per_epoch = []
GNN_Model_training_time_per_epoch = []
def train(EPOCHS, load_index, data):
    GNN_Model_training_loss_per_epoch = []

    for epoch in range(EPOCHS):
        t1 = perf_counter()
        GNN_Model_training_loss, training_acc = train_step(data)
        GNN_Model_training_time_per_epoch.append(perf_counter()-t1)
        print(f'Epoch: {epoch+1:03d}, Model Loss: {GNN_Model_training_loss:.4f}')

        GNN_Model_training_loss_per_epoch.append(GNN_Model_training_loss)
        GNN_Model_training_Acc_per_epoch.append(training_acc)
        #break

        if (epoch + load_index + 1) % 50 == 0 and epoch > 0:
            visualize_losses(GNN_Model_training_loss_per_epoch, epoch + load_index + 1)
        #if (epoch + load_index + 1) % 100 == 0 and epoch > 0:
        #  torch.save({'epoch': epoch+load_index+1, 'model_state_dict': GNN_Model.state_dict(), 'optimizer_state_dict': GNN_Model_Optimizer.state_dict(), 'loss': GNN_Model_training_loss_per_epoch,}, "/content/drive/My Drive/Explainability Methods/" + str(Explainability_name) + " on " + str(Task_name) + "/Model/" + File_Name + str(epoch + load_index + 1)+".pt")



In [ ]:
EPOCHS = 300
load_index = 0

train(EPOCHS, load_index, fake_train_dataloader)

Epoch: 001, Model Loss: 1.1954
Epoch: 002, Model Loss: 1.1402
Epoch: 003, Model Loss: 1.1483
Epoch: 004, Model Loss: 1.1139
Epoch: 005, Model Loss: 1.1787
Epoch: 006, Model Loss: 1.1013
Epoch: 007, Model Loss: 1.0583
Epoch: 008, Model Loss: 1.0322
Epoch: 009, Model Loss: 1.0363
Epoch: 010, Model Loss: 1.0188
Epoch: 011, Model Loss: 1.0504
Epoch: 012, Model Loss: 1.0698
Epoch: 013, Model Loss: 1.0510
Epoch: 014, Model Loss: 1.0194
Epoch: 015, Model Loss: 1.0082
Epoch: 016, Model Loss: 1.0236
Epoch: 017, Model Loss: 1.0089
Epoch: 018, Model Loss: 1.0113
Epoch: 019, Model Loss: 0.9872
Epoch: 020, Model Loss: 1.0139
Epoch: 021, Model Loss: 0.9949
Epoch: 022, Model Loss: 0.9889
Epoch: 023, Model Loss: 1.0009
Epoch: 024, Model Loss: 1.0083
Epoch: 025, Model Loss: 0.9950
Epoch: 026, Model Loss: 0.9714
Epoch: 027, Model Loss: 0.9808
Epoch: 028, Model Loss: 0.9767
Epoch: 029, Model Loss: 0.9541


KeyboardInterrupt: 

In [ ]:
GNN_Model_test_predicted_labels = []
def GNN_Model_test(test_loader):
    GNN_Model.eval()
    correct = 0

    for batch_of_graphs in test_loader:
        if GNN_Model.__class__.__name__ == "GCN_plus_GAP":
            Output_of_Hidden_Layers, pooling_layer_output, ffn_output, soft = GNN_Model(batch_of_graphs)
            GNN_Model_test_pred = soft.argmax(dim=1)
        elif GNN_Model.__class__.__name__ == "DGCNN_Model":
            final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, output_h1, dropout_output_h1, output_h2, soft = GNN_Model(batch_of_graphs, None)
            GNN_Model_test_pred = soft.argmax(dim=1)
        elif GNN_Model.__class__.__name__ == "DIFFPOOL_Model":
            concatination_list_of_poolings, prediction_output_without_soft, soft = GNN_Model(batch_of_graphs, None)
            GNN_Model_test_pred = soft.argmax(dim=1)
        elif GNN_Model.__class__.__name__ == "GIN_Model":
            mlps_output_embeds, mlp_outputs_globalSUMpooled, lin1_output, lin1_output_dropouted, lin2_output, soft = GNN_Model(batch_of_graphs, None)
            GNN_Model_test_pred = soft.argmax(dim=1)

        GNN_Model_test_predicted_labels.append(GNN_Model_test_pred.tolist()[0])
        correct += int((GNN_Model_test_pred == batch_of_graphs.y).sum())
    return correct / len(test_loader.dataset), GNN_Model_test_predicted_labels

In [ ]:
GNN_Model_test_acc, predicted_labels = GNN_Model_test(fake_test_dataloader)
print(f'Test Accuracy: {GNN_Model_test_acc:.4f}')

Test Accuracy: 0.6250


In [ ]:
def power_divergence(X, Y, Z, data_pertubed_Samples, lambda_, significance_level):
    if hasattr(Z, "__iter__"):
        Z = list(Z)
    else:
        raise (f"Z must be an iterable. Got object type: {type(Z)}")

    if (X in Z) or (Y in Z):
        raise ValueError(f"The variables X or Y can't be in Z. Found {X if X in Z else Y} in Z.")
    if len(Z) == 0:
        chi, p_value, dof, expected = stats.chi2_contingency(
            data_pertubed_Samples.groupby([X, Y]).size().unstack(Y, fill_value=0), lambda_=lambda_
        )
    else:
        chi = 0
        dof = 0
        for z_state, df in data_pertubed_Samples.groupby(Z):
            try:
                c, _, d, _ = stats.chi2_contingency(df.groupby([X, Y]).size().unstack(Y, fill_value=0), lambda_=lambda_)
                chi += c
                dof += d
            except ValueError:
                if isinstance(z_state, str):
                    logging.info(f"Skipping the test {X} \u27C2 {Y} | {Z[0]}={z_state}. Not enough samples")
                else:
                    z_str = ", ".join([f"{var}={state}" for var, state in zip(Z, z_state)])
                    logging.info(f"Skipping the test {X} \u27C2 {Y} | {z_str}. Not enough samples")
        p_value = 1 - stats.chi2.cdf(chi, df=dof)
    return chi, p_value, dof

In [ ]:
def cressie_read(X, Y, Z, data_pertubed_Samples, significance_level):
    return power_divergence(X=X, Y=Y, Z=Z, data_pertubed_Samples=data_pertubed_Samples, lambda_="cressie-read", significance_level=significance_level)

In [ ]:
x = [3.01599503e-05, 0.00000000e+00, 2, 3.01599503e-05, 1]
y = np.array(x)
print(y)
z = np.argsort(y)[-3:]
print(z)

[3.01599503e-05 0.00000000e+00 2.00000000e+00 3.01599503e-05
 1.00000000e+00]
[3 4 2]


In [ ]:
if GNN_Model.__class__.__name__ == "GCN_plus_GAP":
    Output_of_Hidden_Layers, pooling_layer_output, ffn_output, soft = GNN_Model(batch_of_graphs)


elif GNN_Model.__class__.__name__ == "DGCNN_Model":
    final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, output_h1, dropout_output_h1, output_h2, softmaxed_h2 = GNN_Model(batch_of_graphs, None)


elif GNN_Model.__class__.__name__ == "DIFFPOOL_Model":
    concatination_list_of_poolings, prediction_output_without_soft, prediction_output = GNN_Model(batch_of_graphs, None)


elif GNN_Model.__class__.__name__ == "GIN_Model":
    mlps_output_embeds, mlp_outputs_globalSUMpooled, lin1_output, lin1_output_dropouted, lin2_output, lin2_output_softmaxed = GNN_Model(batch_of_graphs, None)


In [ ]:
class PGM_Graph_Explainer(object):
    def __init__(self, GNN_Model, graph, perturb_feature_list, perturb_mode, perturb_indicator):

        self.Task_name = 'Graph Classification'
        self.Explainability_name = "PGMExplainer"
        self.GNN_Model = GNN_Model
        self.GNN_Model.eval()

        self.graph = graph
        self.num_layers = 2
        self.perturb_feature_list = perturb_feature_list
        self.perturb_mode = perturb_mode
        self.perturb_indicator = perturb_indicator
        self.node_feat = graph.x.numpy()



    def cressie_read(self, X, Y, Z, data_pertubed_Samples, significance_level):
        return self.power_divergence(X=X, Y=Y, Z=Z, data_pertubed_Samples=data_pertubed_Samples, lambda_="cressie-read", significance_level=significance_level)

    def power_divergence(self, X, Y, Z, data_pertubed_Samples, lambda_, significance_level):
        if hasattr(Z, "__iter__"):
            Z = list(Z)
        else:
            raise (f"Z must be an iterable. Got object type: {type(Z)}")

        if (X in Z) or (Y in Z):
            raise ValueError(f"The variables X or Y can't be in Z. Found {X if X in Z else Y} in Z.")
        if len(Z) == 0:
            chi, p_value, dof, expected = stats.chi2_contingency(
                data_pertubed_Samples.groupby([X, Y]).size().unstack(Y, fill_value=0), lambda_=lambda_
            )
        else:
            chi = 0
            dof = 0
            for z_state, df in data_pertubed_Samples.groupby(Z):
                try:
                    c, _, d, _ = stats.chi2_contingency(df.groupby([X, Y]).size().unstack(Y, fill_value=0), lambda_=lambda_)
                    chi += c
                    dof += d
                except ValueError:
                    if isinstance(z_state, str):
                        logging.info(f"Skipping the test {X} \u27C2 {Y} | {Z[0]}={z_state}. Not enough samples")
                    else:
                        z_str = ", ".join([f"{var}={state}" for var, state in zip(Z, z_state)])
                        logging.info(f"Skipping the test {X} \u27C2 {Y} | {z_str}. Not enough samples")
            p_value = 1 - stats.chi2.cdf(chi, df=dof)
        return chi, p_value, dof


    def perturb_node_features(self, node_feature_matrix, targeted_node_idx, random_perturbation_permission):

        graph_node_features = deepcopy(node_feature_matrix)
        targeted_node_feat_to_perturb_array = deepcopy(graph_node_features[targeted_node_idx])
        # print("targeted_node_feat_to_perturb_array: ", targeted_node_feat_to_perturb_array)
        epsilon = 0.05 * np.max(self.node_feat, axis = 0)

        if random_perturbation_permission == 1:
            for i in range(targeted_node_feat_to_perturb_array.shape[0]):
                if i in self.perturb_feature_list:
                    if self.perturb_mode == "mean":
                        targeted_node_feat_to_perturb_array[i] = np.mean(node_feature_matrix[:,i])
                    elif self.perturb_mode == "zero":
                        targeted_node_feat_to_perturb_array[i] = 0
                    elif self.perturb_mode == "max":
                        targeted_node_feat_to_perturb_array[i] = np.max(node_feature_matrix[:,i])
                    elif self.perturb_mode == "uniform":
                        targeted_node_feat_to_perturb_array[i] = targeted_node_feat_to_perturb_array[i] + np.random.uniform(low=-epsilon[i], high=epsilon[i])
                        if targeted_node_feat_to_perturb_array[i] < 0:
                            targeted_node_feat_to_perturb_array[i] = 0
                        elif targeted_node_feat_to_perturb_array[i] > np.max(self.node_feat, axis = 0)[i]:
                            targeted_node_feat_to_perturb_array[i] = np.max(self.node_feat, axis = 0)[i]

        graph_node_features[targeted_node_idx] = targeted_node_feat_to_perturb_array

        return graph_node_features

    def gather_perturbed_node_features(self, sampling_count, index_to_perturb, noise_offset_percentage, p_value_threshold, class_index):
        if self.GNN_Model.__class__.__name__ == "GCN_plus_GAP":
            Output_of_Hidden_Layers, pooling_layer_output, ffn_output, pred_torch = self.GNN_Model(self.graph)
        elif self.GNN_Model.__class__.__name__ == "DGCNN_Model":
            final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, output_h1, dropout_output_h1, output_h2, pred_torch = self.GNN_Model(self.graph, None)
        elif self.GNN_Model.__class__.__name__ == "DIFFPOOL_Model":
            concatination_list_of_poolings, pred_torch_without_soft, pred_torch = self.GNN_Model(self.graph, None)
        elif GNN_Model.__class__.__name__ == "GIN_Model":
            mlps_output_embeds, mlp_outputs_globalSUMpooled, lin1_output, lin1_output_dropouted, lin2_output, pred_torch = self.GNN_Model(self.graph, None)

        #pred_label = pred_torch.argmax(dim=1)
        pred_label = pred_torch.detach()[:,class_index].to(torch.float32).tolist()[0]

        num_nodes_in_graph = self.node_feat.shape[0]

        Samples = []
        for iteration in range(sampling_count):
            graph_original_features = deepcopy(self.node_feat)
            sample = []
            for node_index in range(num_nodes_in_graph):
                if node_index in index_to_perturb:
                    seed = np.random.randint(100)
                    if seed < noise_offset_percentage:
                        random_perturbation_permission = 1
                        graph_perturbed_features = self.perturb_node_features(node_feature_matrix=graph_original_features, targeted_node_idx=node_index,
                                                                              random_perturbation_permission=random_perturbation_permission)
                    else:
                        random_perturbation_permission = 0
                else:
                    random_perturbation_permission = 0
                sample.append(random_perturbation_permission)

                perturbed_graph = deepcopy(self.graph)
                if random_perturbation_permission:
                    graph_perturbed_features_torch =  torch.tensor(graph_perturbed_features, dtype=torch.float)
                    perturbed_graph.x = graph_perturbed_features_torch

                if self.GNN_Model.__class__.__name__ == "GCN_plus_GAP":
                    Output_of_Hidden_Layers, pooling_layer_output, ffn_output, pred_perturb_torch = self.GNN_Model(perturbed_graph)
                elif self.GNN_Model.__class__.__name__ == "DGCNN_Model":
                    final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, output_h1, dropout_output_h1, output_h2, pred_perturb_torch = self.GNN_Model(perturbed_graph, None)
                elif self.GNN_Model.__class__.__name__ == "DIFFPOOL_Model":
                    concatination_list_of_poolings, pred_torch_without_soft, pred_perturb_torch = self.GNN_Model(perturbed_graph, None)
                elif GNN_Model.__class__.__name__ == "GIN_Model":
                    mlps_output_embeds, mlp_outputs_globalSUMpooled, lin1_output, lin1_output_dropouted, lin2_output, pred_perturb_torch = self.GNN_Model(perturbed_graph, None)


                pred_change = pred_torch.detach()[:,class_index].to(torch.float32).tolist()[0] - pred_perturb_torch.detach()[:,class_index].to(torch.float32).tolist()[0]


                sample.append(pred_change)
            Samples.append(sample)

        Samples = np.asarray(Samples)
        if self.perturb_indicator == "abs":
            Samples = np.abs(Samples)

        top = int(sampling_count/8)
        top_idx = np.argsort(Samples[:, num_nodes_in_graph])[-top:]

        for i in range(sampling_count):
            if i in top_idx:
                Samples[i,num_nodes_in_graph] = 1
            else:
                Samples[i,num_nodes_in_graph] = 0

        return Samples

    def explain(self, num_samples, noise_offset_percentage, top_node, p_value_threshold, class_index):

        if top_node == None:
            top_node = int(self.node_feat.shape[0]/8)

#         Round 1
        Samples = self.gather_perturbed_node_features(sampling_count=num_samples, index_to_perturb=range(self.node_feat.shape[0]),
                                                      noise_offset_percentage=noise_offset_percentage,
                                                      p_value_threshold=p_value_threshold, class_index=class_index)

        data_pertubed_Samples1 = pd.DataFrame(Samples)

        p_values = []
        candidate_nodes = []
        # The entry for the graph classification data is at "num_nodes"
        for node in range(self.node_feat.shape[0]):
            chi2, p, dof = self.cressie_read(X=node, Y=self.node_feat.shape[0], Z=[], data_pertubed_Samples=data_pertubed_Samples1, significance_level=0.05)
            #print("this is returned P: ", p)
            p_values.append(p)


        number_candidates = top_node
        candidate_nodes = np.argpartition(p_values, number_candidates)[0:number_candidates]

#         Round 2
        Samples = self.gather_perturbed_node_features(sampling_count=num_samples, index_to_perturb=candidate_nodes,
                                                      noise_offset_percentage=noise_offset_percentage,
                                                      p_value_threshold=p_value_threshold, class_index=class_index)
        data = pd.DataFrame(Samples)
        #est = ConstraintBasedEstimator(data)

        p_values = []
        dependent_nodes = []

        for node in range(self.node_feat.shape[0]):
            chi2, p, dof = self.cressie_read(X=node, Y=self.node_feat.shape[0], Z=[], data_pertubed_Samples=data, significance_level=0.05)
            #chi2, p = chi_square(node, target, [], data)
            p_values.append(p)
            if p < p_value_threshold:
                dependent_nodes.append(node)


        top_p = np.min((top_node, self.node_feat.shape[0]-1))
        ind_top_p = np.argpartition(p_values, top_p)[0:top_p]
        pgm_nodes = list(ind_top_p)

        return pgm_nodes, p_values, candidate_nodes, dependent_nodes

input_graph = fake_test_dataset[0]
pgmx = PGM_Graph_Explainer(GNN_Model=GNN_Model, graph=input_graph, perturb_feature_list=[None], perturb_mode = "mean",
                           perturb_indicator = "abs")



pgm_nodes, p_values, candidate_nodes, dependent_nodes = pgmx.explain(num_samples=len(input_graph.x), noise_offset_percentage=50,
                                                                     top_node=5, p_value_threshold=0.05, class_index=2)
print("pgm_nodes: ", pgm_nodes, " p_values: ", p_values, " candidate_nodes: ", candidate_nodes, "dependent_nodes: ", dependent_nodes)

pgm_nodes:  [8, 0, 20, 18, 17]  p_values:  [0.4500218182533767, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.4500218182533767, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.4500218182533767, 1.0, 1.0, 1.0, 1.0]  candidate_nodes:  [ 4  0 10 22 16] dependent_nodes:  []


In [ ]:
pgm_nodes_for_each_graph_correct = []
pgm_nodes_for_each_graph_incorrect = []
time_consumption = []

for i in range(len(test_dataset)):

    pgmx = PGM_Graph_Explainer(Model_Name="GCN_plus_GAP", classifier_load_index=200, input_dim=7, hid_dim=7, output_dim=2, graph=test_dataset[i],
                           perturb_feature_list=[None], perturb_mode = "mean", perturb_indicator = "abs")
    start_time = perf_counter()
    pgm_node_correct, p_values_correct, candidate_nodes_correct, dependent_nodes_correct = pgmx.explain(num_samples=len(test_dataset[i].x), percentage=50,
                                                                                                        top_node=3, p_value_threshold=0.05, pred_threshold=0.1,
                                                                                                        ctg='correct')
    pgm_nodes_for_each_graph_correct.append(pgm_node_correct)
    print(pgm_node_correct)
    time_consumption.append(perf_counter() - start_time)
    pgm_node_incorrect, p_values_incorrect, candidate_nodes_incorrect, dependent_nodes_incorrect = pgmx.explain(num_samples=len(test_dataset[i].x), percentage=50,
                                                                                                                top_node=3, p_value_threshold=0.05, pred_threshold=0.1,
                                                                                                                ctg='incorrect')
    pgm_nodes_for_each_graph_incorrect.append(pgm_node_incorrect)
    print(pgm_node_incorrect)



GCN_plus_GAP Input_Dimension: 7
GCN_plus_GAP Hidden_Dimension: 7
GCN_plus_GAP Output_Dimension: 2
GCN_plus_GAP Number_of_Hidden_Layers: 2
eLu is Selected.
[0, 4, 12]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
GCN_plus_GAP Input_Dimension: 7
GCN_plus_GAP Hidden_Dimension: 7
GCN_plus_GAP Output_Dimension: 2
GCN_plus_GAP Number_of_Hidden_Layers: 2
eLu is Selected.
[9, 11, 7]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
GCN_plus_GAP Input_Dimension: 7
GCN_plus_GAP Hidden_Dimension: 7
GCN_plus_GAP Output_Dimension: 2
GCN_plus_GAP Number_of_Hidden_Layers: 2
eLu is Selected.
[9, 11, 7]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
GCN_plus_GAP Input_Dimension: 7
GCN_plus_GAP Hidden_Dimension: 7
GCN_plus_GAP Output_Dimension: 2
GCN_plus_GAP Number_of_Hidden_Layers: 2
eLu is Selected.
[9, 10, 7]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
GCN_plus_GAP Input_Dimension: 7
GCN_plus_GAP Hidden_Dimension: 7
GCN_plus_GAP Output_Dimension: 2
GCN_plus_GAP Number_of_Hidden_Layers: 2
eLu is Selected.
[4, 15, 13]
[

In [ ]:
print(len(time_consumption))
print("time_consumption: ", time_consumption)
print(statistics.mean(time_consumption))

38
time_consumption:  [1.2668439719998332, 0.7257683390000693, 0.5176655889999893, 0.4637203419999878, 1.1856211930000882, 0.8756083120001676, 1.43473268799994, 0.784289211999976, 1.206816420999985, 0.9334678580000855, 0.458032158999913, 1.3268337969998356, 1.5880233769998995, 0.782993728000065, 3.1137604319999355, 0.31455042399988997, 0.8084150800000316, 1.8498538839999128, 0.7707757169998786, 1.612811750999981, 0.7144284099999823, 1.040602447999845, 2.000002936000101, 0.879968295000026, 0.6790533230000619, 0.4538765649999732, 1.4987819080001827, 1.5494426990001102, 0.5338968629998817, 0.7835497840001153, 0.786208839999972, 0.4591743739999856, 0.4762122529998578, 0.44717787499985207, 1.190471634000005, 0.7176257379999242, 0.7385628520000864, 1.7875504660000843]
1.019925566789461


In [ ]:
test_dataset_dropped_correct = deepcopy(test_dataset)
test_dataset_dropped_incorrect = deepcopy(test_dataset)


for i in range(len(pgm_nodes_for_each_graph_correct)):
    for j in range(len(test_dataset_dropped_correct[i].x)):
        if j not in pgm_nodes_for_each_graph_correct[i]:
            test_dataset_dropped_correct[i].x[j] = torch.zeros_like(test_dataset_dropped_correct[i].x[j])

for i in range(len(pgm_nodes_for_each_graph_incorrect)):
    for j in range(len(test_dataset_dropped_incorrect[i].x)):
        if j not in pgm_nodes_for_each_graph_incorrect[i]:
            test_dataset_dropped_incorrect[i].x[j] = torch.zeros_like(test_dataset_dropped_incorrect[i].x[j])


In [ ]:
print(pgm_nodes_for_each_graph_correct[0])
print(test_dataset_dropped_correct[0].x)

[0, 4, 12]
tensor([[1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]])


In [ ]:
Model_Load_iNdex = 1000
GNN_Model = Graph_Network.GCN_plus_GAP(model_name=Model_Name, model_level='graph', input_dim=7, hidden_dim=7, output_dim=2,
                                       num_hid_layers=2, Bias=True, act_fun='ReLu', Weight_Initializer=1, dropout_rate=0.1)
optimizer = torch.optim.Adam(params=GNN_Model.parameters(), lr=0.001, weight_decay=1e-6)
checkpoint = torch.load("/content/drive/My Drive/Explainability Methods/" + str(Explainability_name) + " on " + str(Task_name) + "/Model/" + str(Model_Name) + "_Model_classifier" + str(Model_Load_iNdex)+".pt")
GNN_Model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

GCN_plus_GAP Input_Dimension: 7
GCN_plus_GAP Hidden_Dimension: 7
GCN_plus_GAP Output_Dimension: 2
GCN_plus_GAP Number_of_Hidden_Layers: 2
ReLu is Selected.


In [ ]:
def Compute_ROC_AUC(your_model, your_dataset, masked):
    preds = []
    reals = []
    if masked == False:
        your_model.eval()
        for batched_data in your_dataset:
            #final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, output_h1, dropout_output_h1, output_h2, softmaxed_h2 = your_model(batched_data)
            #Grad_CAM_Test_One_Before_Last_Conv, Grad_CAM_Test_Last_Conv, Grad_CAM_Test_GAP, Grad_CAM_Test_out = your_model(batched_data)
            Output_of_Hidden_Layers, pooling_layer_output, ffn_output, soft = your_model(batched_data)
            #logits = F.log_softmax(Grad_CAM_Test_out, dim=1)
            #prob = F.softmax(logits, dim=1)

            preds.append(soft.cpu().detach())
    else:

        your_model.eval()
        for masked_batch in masked:
            #final_GNN_layer_output, sortpooled_embedings, output_conv1d_1, maxpooled_output_conv1d_1, output_conv1d_2, to_dense, output_h1, dropout_output_h1, output_h2, softmaxed_h2 = your_model(batched_data)
            #Grad_CAM_Test_One_Before_Last_Conv, Grad_CAM_Test_Last_Conv, Grad_CAM_Test_GAP, Grad_CAM_Test_out = your_model(batched_data)
            Output_of_Hidden_Layers, pooling_layer_output, ffn_output, soft = your_model(masked_batch)
            #logits = F.log_softmax(Grad_CAM_Test_out, dim=1)
            #prob = F.softmax(logits, dim=1)

            preds.append(soft.cpu().detach())




    for i, batched_graph in enumerate(your_dataset):
        reals.append(batched_graph.y.cpu().detach().tolist())
    #preds = torch.cat(preds).cpu().numpy()
    #preds = preds[:, 1]
    preds = torch.cat(preds)
    #print(preds)
    preds, max_idxs = torch.max(preds[:], dim=1)
    #print(preds)
    roc_auc = metrics.roc_auc_score(reals, preds, average='macro')
    return roc_auc


def Fidelity_computation(your_model, your_dataset, importance_threshold):
    auc_roc_before_droping_important_nodes = Compute_ROC_AUC(your_model, your_dataset, False)
    print("auc_roc_before_droping_important_nodes: ", auc_roc_before_droping_important_nodes)

    auc_roc_after_droping_important_nodes = Compute_ROC_AUC(your_model, your_dataset, test_dataset_dropped_correct)
    print("auc_roc_after_droping_important_nodes: ", auc_roc_after_droping_important_nodes)

    return auc_roc_before_droping_important_nodes - auc_roc_after_droping_important_nodes
fid0 = Fidelity_computation(GNN_Model, test_dataset, test_dataset_dropped_correct)
fid1 = Fidelity_computation(GNN_Model, test_dataset, test_dataset_dropped_incorrect)
print("Fidelity: ", mean([fid0, fid1]))

auc_roc_before_droping_important_nodes:  0.5505952380952381
auc_roc_after_droping_important_nodes:  0.42857142857142855
auc_roc_before_droping_important_nodes:  0.5505952380952381
auc_roc_after_droping_important_nodes:  0.42857142857142855
Fidelity:  0.12202380952380959


In [ ]:
def crs_subgraph_saliency(your_dataset, masked_data_class0, masked_data_class1):
    binary_scores_class0 = []
    binary_scores_class1 = []
    for graph_main, graph0, graph1 in zip(your_dataset, masked_data_class0, masked_data_class1):
        binary_score_class0 = ''
        binary_score_class1 = ''
        for node_main, node0, node1 in zip(graph_main.x, graph0.x, graph1.x):
            if sum(node0) == 0:
                binary_score_class0 += '1'
            else:
                binary_score_class0 += '0'
            if sum(node1) == 0:
                binary_score_class1 += '1'
            else:
                binary_score_class1 += '0'
        binary_scores_class0.append(binary_score_class0)
        binary_scores_class1.append(binary_score_class1)
    #print("binary_scores_class0: ", binary_scores_class0)
    #print("binary_scores_class1: ", binary_scores_class1)
    return binary_scores_class0, binary_scores_class1

def hamming_distance(string1, string2):

    distance = 0
    L = len(string1)
    for i in range(L):
        if string1[i] != string2[i]:
            distance += 1
    return distance

def compute_contrastivity(your_dataset, masked_data_class0, masked_data_class1):
    binary_scores_class0, binary_scores_class1 = crs_subgraph_saliency(your_dataset, masked_data_class0, masked_data_class1)
    h_dist_list = []
    for cor_bin_scores, incor_bin_scores in zip(binary_scores_class0, binary_scores_class1):
        h_distance = hamming_distance(cor_bin_scores, incor_bin_scores)/len(cor_bin_scores)
        h_dist_list.append(h_distance)
    return statistics.mean(h_dist_list)



crs = compute_contrastivity(test_dataset, test_dataset_dropped_correct, test_dataset_dropped_incorrect)
print("Contrastivity: ", crs)

Contrastivity:  0.7479097046940704


In [ ]:
def spr_subgraph_saliency(masked_data_class0, masked_data_class1):
    binary_scores_class0 = []
    binary_scores_class1 = []
    for graph0, graph1 in zip(masked_data_class0, masked_data_class1):
        binary_score_class0 = []
        binary_score_class1 = []
        for node0, node1 in zip(graph0.x, graph1.x):
            if sum(node0) == 0:
                binary_score_class0.append(1)
            else:
                binary_score_class0.append(0)
            if sum(node1) == 0:
                binary_score_class1.append(1)
            else:
                binary_score_class1.append(0)
        binary_scores_class0.append(binary_score_class0)
        binary_scores_class1.append(binary_score_class1)

    return binary_scores_class0, binary_scores_class1

def compute_sparsity(masked_data_class0, masked_data_class1):
    binary_scores_class0, binary_scores_class1 = spr_subgraph_saliency(masked_data_class0, masked_data_class1)

    sparsity_list = []
    for cor_binary_score, incor_binary_score in zip(binary_scores_class0, binary_scores_class1):
        sparsity = 1 - ((sum(cor_binary_score) + sum(incor_binary_score))/(2*len(incor_binary_score)))
        sparsity_list.append(sparsity)

    return statistics.mean(sparsity_list)



spr = compute_sparsity(test_dataset_dropped_correct, test_dataset_dropped_incorrect)
print("Sparsity: ", spr)

Sparsity:  0.5637965676345319


In [ ]:
task_dict = {"Graph Classification": "GC", "Node Classification": "NC"}
l = [Explainability_name, task_dict[str(Task_name)], "MUTAG", "GCN+GAP", Auc_Roc, Auc_Prc, GNN_Model_training_Acc_per_epoch[-1], Acc, Acc2, fid, crs, spr, statistics.mean(time_consumption), statistics.mean(GNN_Model_training_time_per_epoch), samples_time]

In [ ]:
task_dict = {"Graph Classification": "GC", "Node Classification": "NC"}
with open("/content/drive/My Drive/Explainability Methods/Evaluation of Explicability Methods/Experimental Results/Comparisons_ExMethods_Final_Format.csv", 'a') as outcsv:
    wr = csv.writer(outcsv, dialect='excel', delimiter=',')
    wr.writerow(l)